In [1]:
library(pheatmap)
library(plyr)

NameError: name 'library' is not defined

In [ ]:
# Load integrals, peak ranges, and sample data# Load  
integrals = read.table('integrals_20170323.txt', header=FALSE,
                         sep = ",")
integral_ranges = read.table('coculture_peak_ranges.txt',header=TRUE,
                             sep = ",")
master <- read.table(file="merged_metadata.txt",header=TRUE,sep='\t')

In [ ]:
# set column names of integrals using integral_ranges met names
colnames(integrals) <- integral_ranges$met_name
mets = colnames(integrals)
# Set rownames using experiment_id. This should be ordered correctly.
integrals$experiment_id <- master$experiment_id
# merge so that classes and integrals are in one dataframe
all_data <- join(master,integrals,by="experiment_id")

In [ ]:
# remove non-growing samples from the data
# 500 and 492 did not grow in experiment 3
# 502 did not grow in experiment 2 or 3
all_data = all_data[!(all_data$Run == 2 & grepl('502',all_data$species)),]
all_data = all_data[!(all_data$Run == 3 & grepl('502',all_data$species)),]
all_data = all_data[!(all_data$Run == 3 & grepl('500',all_data$species)),]
all_data = all_data[!(all_data$Run == 3 & grepl('492',all_data$species)),]
# remove experiment 4 (3-species subset including only 356,361,519)
all_data = all_data[all_data$Run != "4",]

In [ ]:
# replace negative values in integrals with 0.
all_data[,colnames(all_data) %in% mets] = 
        (apply(all_data[,colnames(all_data) %in% mets],2,function(x){x[x<0] = 0;x}))
# center the data by subtracting the mean of blanks
blanks = subset(all_data,species == '0')
blank_means = colMeans(blanks[,names(blanks) %in% mets])
centered_integrals <- sweep(all_data[,mets], 2, blank_means, `-`)
all_data[,mets] = centered_integrals

In [ ]:
known_mets known_m = mets[-grep("unknown", mets)]
monoculture_only = all_data[all_data$species %in% c('356','360','361','492','500','519'),]
# collapse columns into means
mono356 = colMeans(monoculture_only[monoculture_only$species == '356',known_mets])
mono360 = colMeans(monoculture_only[monoculture_only$species == '360',known_mets])
monoculture_for_plotting = rbind(mono356,mono360)
mono361 = colMeans(monoculture_only[monoculture_only$species == '361',known_mets])
monoculture_for_plotting = rbind(monoculture_for_plotting,mono361)
mono492 = colMeans(monoculture_only[monoculture_only$species == '492',known_mets])
monoculture_for_plotting = rbind(monoculture_for_plotting,mono492)
mono500 = colMeans(monoculture_only[monoculture_only$species == '500',known_mets])
monoculture_for_plotting = rbind(monoculture_for_plotting,mono500)
# Exclude 502 for the paper
#mono502 = colMeans(monoculture_only[monoculture_only$species == '502',known_mets])
#monoculture_for_plotting = rbind(monoculture_for_plotting,mono502)
mono519 = colMeans(monoculture_only[monoculture_only$species == '519',known_mets])
monoculture_for_plotting = rbind(monoculture_for_plotting,mono519)
row.names(monoculture_for_plotting) = c('ASF356','ASF360','ASF361','ASF492','ASF500','ASF519')
# try min/max scaling within each column
monoculture_for_plotting_maxnorm = sweep(monoculture_for_plotting,MARGIN=2,apply(monoculture_for_plotting,2,function(x) max(abs(x), na.rm = TRUE)), FUN="/")


In [ ]:
# generate a heatmap of monoculture metabolomes using the centered, max/min normalized integrals

pheatmap(monoculture_for_plotting_maxnorm,filename='~/Documents/Projects/asf_interactions/results/monoculture_metabolome.jpg',width=9,height=3.2,cluster_rows=FALSE)